# Journey of a pod


What happens when I run `kubectl run nginx --image nginx`


## State of the cluster before

* What's in etcd ?
* Are there any containers running on any of the nodes ?
* What's on the network stack?

## Running the command

* kubectl sends a http call to the api-server with this payload ...
* the api server ...
* the controll server ...
* the scheduler ...
* ... calls kubelet on one of the two nodes
* the node runs the container
    * image is pulled and stored in ...
    * ... runs the container with ...
* kubelt reports back to the api-server
* What's new in the network ?

## Challenges

It might be complicated to get kubernetes system logs [https://kubernetes.io/docs/concepts/cluster-administration/system-traces/](https://kubernetes.io/docs/concepts/cluster-administration/system-traces/)

## Links

* [Kubernetes components (kubernetes.io)](https://kubernetes.io/docs/concepts/overview/components/)
* [Discover Kubernetes API Calls from kubectl (trstringer.com)](https://trstringer.com/kubernetes-api-call-from-kubectl/)
* [Access Clusters Using the Kubernetes API (kubernetes.io)](https://kubernetes.io/docs/tasks/administer-cluster/access-cluster-api/#without-kubectl-proxy)

## State of the cluster before

In [53]:
# What's running in the default namespace ?
! kubectl get all

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.233.0.1   <none>        443/TCP   121m


In [54]:
# What's in that service/kubernetes ?
! kubectl describe service/kubernetes

Name:              kubernetes
Namespace:         default
Labels:            component=apiserver
                   provider=kubernetes
Annotations:       <none>
Selector:          <none>
Type:              ClusterIP
IP Family Policy:  SingleStack
IP Families:       IPv4
IP:                10.233.0.1
IPs:               10.233.0.1
Port:              https  443/TCP
TargetPort:        6443/TCP
Endpoints:         xxx.xxx.235.216:6443
Session Affinity:  None
Events:            <none>


[Publishing Services (ServiceTypes)](https://kubernetes.io/docs/concepts/services-networking/service/#publishing-services-service-types)

> ClusterIP: Exposes the Service on a cluster-internal IP. Choosing this value makes the Service only reachable from within the cluster. This is the default ServiceType.

It seems like `service/kubernetes` allows anyone within the cluster to talk to the api-server.

What if we try to make a call from within one of the nodes ?

In [17]:
! ssh -q node2 "curl https://kubernetes.default.svc.cluster.local:6443/api/v1/namespaces/default/pods?limit=500"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to kubernetes.default.svc.cluster.local port 6443: Connection refused


Hypothesis: The connection was refused because the call was not authenticated.

In [52]:
# Which containers are running on node 2 ?
! ssh -q node2 "nerdctl ps"

CONTAINER ID    IMAGE                                       COMMAND                   CREATED        STATUS    PORTS    NAMES
03d415712a9f    k8s.gcr.io/kube-proxy:v1.23.7               "/usr/local/bin/kube…"    2 hours ago    Up                 k8s://kube-system/kube-proxy-xn9rx/kube-proxy                                         
0c12bf085273    k8s.gcr.io/pause:3.3                        "/pause"                  2 hours ago    Up                 k8s://kube-system/nodelocaldns-f29kg                                                  
143622dbd6d7    k8s.gcr.io/pause:3.3                        "/pause"                  2 hours ago    Up                 k8s://kube-system/nginx-proxy-node2                                                   
17b03d491dad    k8s.gcr.io/dns/k8s-dns-node-cache:1.21.1    "/node-cache -locali…"    2 hours ago    Up                 k8s://kube-system/nodelocaldns-f29kg/node-cache                                       
1d23a60e2b99    quay.io/calico/kube-controller

In [55]:
# Which containers are running on node 3 ?
! ssh -q node3 "nerdctl ps"

CONTAINER ID    IMAGE                                               COMMAND                   CREATED        STATUS    PORTS    NAMES
1210821f8181    k8s.gcr.io/dns/k8s-dns-node-cache:1.21.1            "/node-cache -locali…"    2 hours ago    Up                 k8s://kube-system/nodelocaldns-s728x/node-cache                                            
4b9c1c7a61af    k8s.gcr.io/pause:3.3                                "/pause"                  2 hours ago    Up                 k8s://kube-system/calico-node-h47zr                                                        
56abc426aabc    k8s.gcr.io/pause:3.3                                "/pause"                  2 hours ago    Up                 k8s://kube-system/kube-proxy-b9gh2                                                         
8a92fd9a6409    docker.io/library/nginx:1.21.4                      "/docker-entrypoint.…"    2 hours ago    Up                 k8s://kube-system/nginx-proxy-node3/nginx-proxy                               

## Running a pod

In [ ]:
! kubectl run nginx --image=nginx --restart=Never